# Generate FWE Corrected R-Squared Map 
- Using maximum statistic correction
- Notes on controlling a regression:
    - Adding covariates to a regression will 'control' for them, but will almost always increase the R-squared. 
    - To 'remove' a covariate from the regression, you will want to regress a nuisance covariate OUT of the covariate of interest. 
        - This means your regressor will become the residuals from the regression of cov_1 ~ nuisance_cov1

Import Niftis
- These are EXPECTED to have subject IDs which are IDENTICAL to the subject IDs that go in the covarite DF column names below
- This is expected to ultimately have the form:

|        | Subject 1 | Subject 2 | Subject 3 | Subject 4 | Subject 5 | Subject 6 | Subject 7 | Subject 8 | Subject 9 | Subject 10 | ... | Subject 40 | Subject 41 | Subject 42 | Subject 43 | Subject 45 | Subject 46 | Subject 47 | Subject 48 | Subject 49 | Subject 50 |
|----------|------------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|------------|-----|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|
| Voxel 1     | 3          | 4         | 7         | 2         | 2         | 2         | 9         | 4         | 7         | 5          | ... | 5           | 2           | 7           | 7           | 3           | 8           | 8           | 1           | 1           | 3           |
| . . .      | ...         | ...        | ...         | ...         | ...         | ...         | ...         | ...         | ...         | ...          | ... | ...           | ...           | ...           | ...           | ...           | ...           | 7           | ...           | ...           | ...           |
| Voxel N     | 2          | 1         | 0         | 1         | 3         | 4         | 9         | 5         | 8         | 6          | ... | 6           | 3           | 8           | 8           | 4           | 9           | 9           | 2           | 2           | 4           |

In [19]:
import_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/collaborations/hart_epilepsy_meta_analysis/SeedsYeo'
file_target = '*'
file_column = None

In [ ]:
from calvin_utils.file_utils.import_functions import GiiNiiFileImport
giinii = GiiNiiFileImport(import_path=import_path, file_column=file_column, file_pattern=file_target)
nimg_df = giinii.run()
nimg_df

Fix names

In [40]:
pre = 'SeedsYeo/'
post = ''

In [ ]:
nimg_df = GiiNiiFileImport.splice_colnames(nimg_df, pre, post)
nimg_df


Import Covariates
- These are EXPECTED to have subject IDs which are IDENTICAL to the subject IDs that go in the neuroimaging DF column names above
- Do NOT enter strings for categorical variables. All data must be numerical
- **The CSV is expected to be in this format**

```
+-----+----------------------------+--------------+--------------+--------------+
| ID  | Nifti_File_Path            | Covariate_1  | Covariate_2  | Covariate_3  |
+-----+----------------------------+--------------+--------------+--------------+
| 1   | /path/to/file1.nii.gz      | 0.5          | 1.2          | 3.4          |
| 2   | /path/to/file2.nii.gz      | 0.7          | 1.4          | 3.1          |
| 3   | /path/to/file3.nii.gz      | 0.6          | 1.5          | 3.5          |
| 4   | /path/to/file4.nii.gz      | 0.9          | 1.1          | 3.2          |
| ... | ...                        | ...          | ...          | ...          |
+-----+----------------------------+--------------+--------------+--------------+
```

In [42]:
input_csv_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/Kahana_Epilepsy_iEEG/memory_tms_stim_data.csv'

In [ ]:
import pandas as pd
data_df = pd.read_csv(input_csv_path, index_col=0)
data_df

In [44]:
# import numpy as np
# data_df = data_df[data_df['StandardDeviation'] != 0]
# data_df['inverse_variance'] = 1/((data_df['StandardDeviation']/100)**2)
# data_df['weighted_outcome'] = data_df['Outcome']*data_df['inverse_variance']
# data_df

Choose Rows to Keep

In [45]:
col_to_keep_list = ['Outcome', 'subject', 'Generalized', 'Focal', 'Multifocal', 'Multiple', 'CertaintyWeight']

Extra

 Should Now look like: 
- The extra rows will be used as covariates
 
|        | Subject 1 | Subject 2 | Subject 3 | Subject 4 | Subject 5 | Subject 6 | Subject 7 | Subject 8 | Subject 9 | Subject 10 | ... | Subject 40 | Subject 41 | Subject 42 | Subject 43 | Subject 45 | Subject 46 | Subject 47 | Subject 48 | Subject 49 | Subject 50 |
|----------|------------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|------------|-----|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|
| Var1     | 3          | 4         | 7         | 2         | 2         | 2         | 9         | 4         | 7         | 5          | ... | 5           | 2           | 7           | 7           | 3           | 8           | 8           | 1           | 1           | 3           |
| Var2     | 45         | 32        | 1         | 0         | 2         | 3         | 8         | 3         | 6         | 4          | ... | 4           | 1           | 6           | 6           | 2           | 7           | 7           | 0           | 0           | 2           |
| Var3     | 2          | 1         | 0         | 1         | 3         | 4         | 9         | 5         | 8         | 6          | ... | 6           | 3           | 8           | 8           | 4           | 9           | 9           | 2           | 2           | 4           |


In [ ]:
data_df= data_df.loc[:, col_to_keep_list].T
data_df.columns = data_df.loc['subject', :]
data_df = data_df.drop('subject')
data_df.dropna(inplace=True, axis=1)
data_df

Name of the row with outcomes
- If None, will regress covariates on voxels. 
- If you are interested in how voxel values relate to an outcome, make sure you have this row in your DF and name it below. 

In [47]:
outcome_row = None

Is there a particular mask you want to use?
- MUST match the resolution of voxelwise data being analyzed. 
- If you set None, the voxelwise data will be used for thresholding. 
    - Values below mask_threshold (float) will be set to 0. 
- Warning: bad masking may result in failed experiments. Erroneous voxels outside the brain will influence the correction. 

In [48]:
mask_path = '/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii'
mask_threshold = 0

Choose Max Stat Correction Method
- None | pseudo_var_smooth | var_smooth
- None performs voxelwise max-stat FWE correction, which is the most stringent FWE correction right after Bonferroni. 

In [49]:
max_stat_method = None

Where do you want to save the results to?

In [50]:
out_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/collaborations/hart_epilepsy_meta_analysis/Figures/regression/1k-unsmoothed-outcome~certainty+generalized+focal+multifocal+multiple+voxel/commutated'

In [ ]:
from calvin_utils.permutation_analysis_utils.lin_reg_fwe import CalvinFWEMap
calvin_fwe = CalvinFWEMap(neuroimaging_dataframe=nimg_df, variable_dataframe=data_df, 
                          mask_threshold=mask_threshold, 
                          mask_path=mask_path,
                          max_stat_method=max_stat_method,
                          outcome_row=outcome_row,
                          out_dir=out_dir,
                          vectorize=True)
calvin_fwe.run(n_permutations=1000, debug=False)

Visualize the FWE Corrected Image

In [ ]:
calvin_fwe.corrected_img

Visualize the P-Values (FWE Corrected) Used to Correct the Above

In [ ]:
calvin_fwe.p_img

Visualize the Uncorrected Image

In [ ]:
calvin_fwe.uncorrected_img

Visualize t values of last coefficient

In [ ]:
calvin_fwe.final_t_value